In [97]:
import numpy as np
import torch
import torch.nn as nn
import os
import scipy.sparse
import pandas as pd
from tqdm import tqdm
from random import *
import copy
import pickle

In [58]:
data_root = os.environ['KakaoMelonArenaHome']

In [62]:
def lil_to_tensor(lil_mtrx):
    coo_mtrx = lil_mtrx.tocoo()
    values = torch.FloatTensor(coo_mtrx.data)
    indices = torch.LongTensor(np.vstack((coo_mtrx.row, coo_mtrx.col)))
    shape = torch.Size(coo_mtrx.shape)

    X = torch.sparse.FloatTensor(indices, values, shape)
    return X

In [85]:
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        #self.encoder = nn.Sequential(nn.Linear(N_SONGS, 1))
        #self.decoder = nn.Sequential(nn.Linear(1, N_SONGS))
        self.encoder = nn.Sequential(nn.Linear(N_SONGS, 512), nn.Linear(512, 128), nn.Linear(128, 32))
        self.decoder = nn.Sequential(nn.Linear(32, 128), nn.Linear(128, 512), nn.Linear(512, N_SONGS))
        #self.encoder = nn.Sequential(nn.Linear(N_SONGS, 2048), nn.Linear(2048, 512), nn.Linear(512, 128), nn.Linear(128, 32))
        #self.decoder = nn.Sequential(nn.Linear(32, 128), nn.Linear(128, 512), nn.Linear(512, 2048), nn.Linear(2048, N_SONGS))
    
    def forward(self, X):
        out = self.encoder(X)
        out = self.decoder(out)
        return out

In [86]:
def add_noise(lil_mtrx, p):
    noised_mtrx = copy.deepcopy(lil_mtrx)
    n = noised_mtrx.getnnz()
    n_noises = int(n * p)
    indexes = [randrange(n) for i in range(n_noises)]
    rows, cols = noised_mtrx.nonzero()
    for i in range(n_noises):
        rand = randrange(n)
        i, j = rows[rand], cols[rand]
        noised_mtrx[i, j] = 0
    return noised_mtrx
    

In [94]:
def train(lil_mtrx, batch_size, n_epoch, lr, save_path, load_epoch = 0):
    model = AutoEncoder()
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = lr)
    
    if(load_epoch != 0):
        checkpoint = torch.load(save_path + '/' + str(load_epoch) + '.pth')
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    n_plylsts, n_songs = lil_mtrx.shape
    n_batches = int(np.ceil(n_plylsts / batch_size))
    for epoch in range(load_epoch, n_epoch):
        print('---------- train at epoch %d ----------'%epoch)
        loss_sum = 0
        for b in range(n_batches):
            if b == n_batches - 1:
                mtrx = lil_mtrx[b * batch_size :, :]
                noised_mtrx = add_noise(mtrx, 0.2)
                batch = lil_to_tensor(noised_mtrx)
                gt = lil_to_tensor(mtrx)
            else:
                mtrx = lil_mtrx[b * batch_size : (b + 1) * batch_size, :]
                noised_mtrx = add_noise(mtrx, 0.2)
                batch = lil_to_tensor(noised_mtrx)
                gt = lil_to_tensor(mtrx)
            
            optimizer.zero_grad()
            pred = model(batch)
            loss = criterion(pred, gt.to_dense())
            loss.backward()
            optimizer.step()
            
            print('batch %d / %d | loss: %.9f'%(b, n_batches, loss))
            loss_sum += loss
        print('*** loss avg at epoch %d: %.9f'%(epoch, loss_sum / n_batches))
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, save_path + '/' + str(epoch) + '.pth')


In [95]:
SAVE_PATH = data_root + '/resume'
train(lil_mtrx, 32, 100, 0.01, SAVE_PATH)

---------- train at epoch 0 ----------
batch 0 / 2877 | loss: 0.002772496


KeyboardInterrupt: 